In [1]:
# preliminary
from tensorflow.keras.datasets import imdb
from tensorflow import keras
from tensorflow.keras import layers 
import numpy as np
import tensorflow as tf

In [2]:
# load imdb data
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

In [3]:
# 
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

train_data = vectorize_sequences(train_data)

# define model
def build_model():
    model = keras.Sequential([
        layers.Dense(4, activation="relu"),
        layers.Dense(4, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

In [4]:
print(train_labels)

[1 0 0 ... 0 1 0]


In [5]:
class RootMeanSquaredError(keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)
    
    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))
    
    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [6]:
class F1Score(keras.metrics.Metric):
    def __init__(self, name="F1score_user", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.predicted_positives = self.add_weight(name="predicted_positives", initializer="zeros") 
        self.actual_positives = self.add_weight(name="actual_positives", initializer="zeros")
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))

        actual_positives = tf.reduce_sum(y_true)
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        predicted_positives = tf.reduce_sum(y_pred_classes)
        
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)     
        self.actual_positives.assign_add(actual_positives)     
    
    def result(self):
        precision = ( tf.cast(self.true_positives, tf.float32) 
              / tf.cast(self.predicted_positives, tf.float32) + tf.keras.backend.epsilon())
        recall = ( tf.cast(self.true_positives, tf.float32) 
              / tf.cast(self.actual_positives, tf.float32) + tf.keras.backend.epsilon())
        return  (2 * recall * precision) / (recall + precision + tf.keras.backend.epsilon())
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.predicted_positives.assign(0)
        self.actual_positives.assign(0)
                

In [7]:
class acc_user(keras.metrics.Metric):
    def __init__(self, name="acc_user", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.predicted_positives = self.add_weight(name="predicted_positives", initializer="zeros") 
        self.actual_positives = self.add_weight(name="actual_positives", initializer="zeros")
        self.true_negatives = self.add_weight(name="true_negatives", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )
#         tp + tn / tp + fp + tn + fn = acc
#         tp + fp = p.p
#         tp + fn = a.p
#         tn             
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))
               
        actual_positives = tf.reduce_sum(y_true)
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        predicted_positives = tf.reduce_sum(y_pred_classes)
        # false_negatives = tf.reduce_sum(y_true.shape[0] - predicted_positives - (y_true.shape[0]-y_true))
        # all samples - p.p - tn
        # tn = !y_true
        #true_negatives = tf.reduce_sum(tf.constant(1, dtype="float32") -y_true)
        true_negatives = tf.reduce_sum((1 -y_true)*(1-y_pred_classes))
        total_samples = tf.shape(y_pred)[0]
        
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)     
        self.actual_positives.assign_add(actual_positives)     
        self.true_negatives.assign_add(true_negatives)
        self.total_samples.assign_add(total_samples)    
        
    def result(self):

        return  (self.true_positives+self.true_negatives)/(tf.cast(self.total_samples, tf.float32) + tf.keras.backend.epsilon())
        #return self.total_samples
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.predicted_positives.assign(0)
        self.actual_positives.assign(0)
        self.true_negatives.assign(0)
        self.total_samples.assign(0)
                

In [8]:
class tp(keras.metrics.Metric):
    def __init__(self, name="tp", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.predicted_positives = self.add_weight(name="predicted_positives", initializer="zeros") 
        self.actual_positives = self.add_weight(name="actual_positives", initializer="zeros")
        self.true_negatives = self.add_weight(name="true_negatives", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )
#         tp + tn / tp + fp + tn + fn = acc
#         tp + fp = p.p
#         tp + fn = a.p
#         tn 
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))
        
        
        actual_positives = tf.reduce_sum(y_true)
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        predicted_positives = tf.reduce_sum(y_pred_classes)
        # false_negatives = tf.reduce_sum(y_true.shape[0] - predicted_positives - (y_true.shape[0]-y_true))
        # all samples - p.p - tn
        # tn = !y_true
        true_negatives = tf.reduce_sum((1 -y_true)*(1-y_pred_classes))
        total_samples = tf.shape(y_pred)[0]
        
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)     
        self.actual_positives.assign_add(actual_positives)     
        self.true_negatives.assign_add(true_negatives)
        self.total_samples.assign_add(total_samples)    
        
    def result(self):

        #return  (self.true_positives+self.true_negatives)/(tf.cast(self.total_samples, tf.float32) + tf.keras.backend.epsilon())
        return self.true_positives
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.predicted_positives.assign(0)
        self.actual_positives.assign(0)
        self.true_negatives.assign(0)
        self.total_samples.assign(0)
                

In [9]:
class tn(keras.metrics.Metric):
    def __init__(self, name="tn", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.predicted_positives = self.add_weight(name="predicted_positives", initializer="zeros") 
        self.actual_positives = self.add_weight(name="actual_positives", initializer="zeros")
        self.true_negatives = self.add_weight(name="true_negatives", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )
#         tp + tn / tp + fp + tn + fn = acc
#         tp + fp = p.p
#         tp + fn = a.p
#         tn 
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))
                
        actual_positives = tf.reduce_sum(y_true)
        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        predicted_positives = tf.reduce_sum(y_pred_classes)
        # false_negatives = tf.reduce_sum(y_true.shape[0] - predicted_positives - (y_true.shape[0]-y_true))
        # all samples - p.p - tn
        # tn = !y_true
        true_negatives = tf.reduce_sum((1 -y_true)*(1-y_pred_classes))
        total_samples = tf.shape(y_pred)[0]
        
        self.true_positives.assign_add(true_positives)
        self.predicted_positives.assign_add(predicted_positives)     
        self.actual_positives.assign_add(actual_positives)     
        self.true_negatives.assign_add(true_negatives)
        self.total_samples.assign_add(total_samples)    
        
    def result(self):

        #return  (self.true_positives+self.true_negatives)/(tf.cast(self.total_samples, tf.float32) + tf.keras.backend.epsilon())
        return self.true_negatives
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.predicted_positives.assign(0)
        self.actual_positives.assign(0)
        self.true_negatives.assign(0)
        self.total_samples.assign(0)
                

In [10]:
class F1Score2(keras.metrics.Metric):
    def __init__(self, name="F1score_user2", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.true_negatives = self.add_weight(name="true_negatives", initializer="zeros") 
        self.false_positives = self.add_weight(name="false_positives", initializer="zeros")
        self.false_negatives = self.add_weight(name="false_negatives", initializer="zeros")
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))

        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        true_negatives = tf.reduce_sum((1 -y_true)*(1-y_pred_classes))
        false_positives = tf.reduce_sum(tf.cast(((y_pred_classes == 1) & (y_true == 0)), tf.float32))
        false_negatives = tf.reduce_sum(tf.cast( ((y_pred_classes == 0) & (y_true == 1)), tf.float32))
        
        self.true_positives.assign_add(true_positives)
        self.true_negatives.assign_add(true_negatives)     
        self.false_positives.assign_add(false_positives)     
        self.false_negatives.assign_add(false_negatives)     
    
    def result(self):
        precision = tf.cast(self.true_positives, "float32")/ (tf.cast(self.true_positives, "float32")+ tf.cast(self.false_positives, "float32")+ tf.keras.backend.epsilon())
                
        recall = tf.cast(self.true_positives, "float32")/ (tf.cast(self.true_positives, "float32")+ tf.cast(self.false_negatives, "float32")+ tf.keras.backend.epsilon())
                
        return  (2 * recall * precision) / (recall + precision + tf.keras.backend.epsilon())
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.true_negatives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)      

In [11]:
class fp2(keras.metrics.Metric):
    def __init__(self, name="fp2", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.true_negatives = self.add_weight(name="true_negatives", initializer="zeros") 
        self.false_positives = self.add_weight(name="false_positives", initializer="zeros")
        self.false_negatives = self.add_weight(name="false_negatives", initializer="zeros")
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))

        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        true_negatives = tf.reduce_sum((1 -y_true)*(1-y_pred_classes))
        false_positives = tf.reduce_sum(tf.cast( ((y_pred_classes == 1) & (y_true == 0)), tf.float32))
        false_negatives = tf.reduce_sum(tf.cast( ((y_pred_classes == 0) & (y_true == 1)), tf.float32))
        
        self.true_positives.assign_add(true_positives)
        self.true_negatives.assign_add(true_negatives)     
        self.false_positives.assign_add(false_positives)     
        self.false_negatives.assign_add(false_negatives)
    
    def result(self):
        return  self.false_positives
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.true_negatives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)      

In [12]:
class fn2(keras.metrics.Metric):
    def __init__(self, name="fn2", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="true_positives", initializer="zeros")
        self.true_negatives = self.add_weight(name="true_negatives", initializer="zeros") 
        self.false_positives = self.add_weight(name="false_positives", initializer="zeros")
        self.false_negatives = self.add_weight(name="false_negatives", initializer="zeros")
            
    def update_state(self, y_true, y_pred, sample_weight=None):
        # For binary classification, threshold predictions at 0.5
        y_pred_classes = tf.squeeze(tf.cast(y_pred > 0.5, "float32"))
        y_true = tf.squeeze(tf.cast(y_true, "float32"))

        true_positives = tf.reduce_sum(y_true * y_pred_classes)
        true_negatives = tf.reduce_sum((1 -y_true)*(1-y_pred_classes))
        false_positives = tf.reduce_sum(tf.cast( ((y_pred_classes == 1) & (y_true == 0)), tf.float32))
        false_negatives = tf.reduce_sum(tf.cast( ((y_pred_classes == 0) & (y_true == 1)), tf.float32))
        
        self.true_positives.assign_add(true_positives)
        self.true_negatives.assign_add(true_negatives)     
        self.false_positives.assign_add(false_positives)     
        self.false_negatives.assign_add(false_negatives)     
    
    def result(self):

        return  self.false_negatives
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.true_negatives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)      

In [13]:
# define model
model_new = build_model()

In [14]:
model_new.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                 metrics=["accuracy", acc_user(), F1Score(), F1Score2(), tp(), tn(), fp2(), fn2()])

In [15]:
history_new = model_new.fit(
    train_data, train_labels,
    epochs=20, batch_size=64, validation_split=0.4
)

Epoch 1/20
235/235 [==============================] - 7s 13ms/step - loss: 0.5284 - accuracy: 0.8097 - acc_user: 0.8097 - F1score_user: 0.7950 - F1score_user2: 0.7950 - tp: 5535.0000 - tn: 6611.0000 - fp2: 856.0000 - fn2: 1998.0000 - val_loss: 0.3788 - val_accuracy: 0.8745 - val_acc_user: 0.8745 - val_F1score_user: 0.8737 - val_F1score_user2: 0.8737 - val_tp: 4339.0000 - val_tn: 4406.0000 - val_fp2: 627.0000 - val_fn2: 628.0000
Epoch 2/20
235/235 [==============================] - 2s 7ms/step - loss: 0.2894 - accuracy: 0.9025 - acc_user: 0.9025 - F1score_user: 0.9034 - F1score_user2: 0.9034 - tp: 6843.0000 - tn: 6694.0000 - fp2: 773.0000 - fn2: 690.0000 - val_loss: 0.2820 - val_accuracy: 0.8908 - val_acc_user: 0.8908 - val_F1score_user: 0.8911 - val_F1score_user2: 0.8911 - val_tp: 4466.0000 - val_tn: 4442.0000 - val_fp2: 591.0000 - val_fn2: 501.0000
Epoch 3/20
235/235 [==============================] - 2s 7ms/step - loss: 0.2114 - accuracy: 0.9256 - acc_user: 0.9256 - F1score_user: 0.9

235/235 [==============================] - 2s 7ms/step - loss: 0.0334 - accuracy: 0.9897 - acc_user: 0.9897 - F1score_user: 0.9898 - F1score_user2: 0.9898 - tp: 7459.0000 - tn: 7387.0000 - fp2: 80.0000 - fn2: 74.0000 - val_loss: 0.5932 - val_accuracy: 0.8650 - val_acc_user: 0.8650 - val_F1score_user: 0.8634 - val_F1score_user2: 0.8634 - val_tp: 4266.0000 - val_tn: 4384.0000 - val_fp2: 649.0000 - val_fn2: 701.0000


In [16]:
# history_new = model_new.fit(
#     train_data, train_labels,
#     epochs=20, batch_size=4, validation_split=0.4
# )

In [17]:
a = tf.constant([[0.5],[1],[1]], dtype="float32")
a.shape[0]

3

In [18]:
1 - a

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.5],
       [0. ],
       [0. ]], dtype=float32)>

In [19]:
s = tf.reduce_sum(a)

In [20]:
s

<tf.Tensor: shape=(), dtype=float32, numpy=2.5>

In [21]:
b = tf.constant([0, 1, 0, 1, 1, 1, 0])
c = tf.constant([0, 1, 1, 1, 1, 0, 1])

b & c

<tf.Tensor: shape=(7,), dtype=int32, numpy=array([0, 1, 0, 1, 1, 0, 0], dtype=int32)>

In [22]:
d = tf.constant([False, True, False, True, True, True, False], dtype=bool)
e = tf.constant([False, True, True, True, True, False, True], dtype=bool)
d & e

<tf.Tensor: shape=(7,), dtype=bool, numpy=array([False,  True, False,  True,  True, False, False])>

In [23]:
#tf.reduce_sum(tf.cast( ((y_pred_classes == 1) and (y_true == 0)), tf.float32))
d & e

<tf.Tensor: shape=(7,), dtype=bool, numpy=array([False,  True, False,  True,  True, False, False])>

In [24]:
(c == 1) 

<tf.Tensor: shape=(7,), dtype=bool, numpy=array([False,  True,  True,  True,  True, False,  True])>

In [25]:
(b == 0)

<tf.Tensor: shape=(7,), dtype=bool, numpy=array([ True, False,  True, False, False, False,  True])>

In [26]:
(c == 1) & (b == 0)

<tf.Tensor: shape=(7,), dtype=bool, numpy=array([False, False,  True, False, False, False,  True])>

In [27]:
!pip list

Package                       Version
----------------------------- ------------------
absl-py                       0.15.0
aiohttp                       3.8.1
aiosignal                     1.2.0
albumentations                1.1.0
antlr4-python3-runtime        4.8
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.0.5
astunparse                    1.6.3
async-generator               1.10
async-timeout                 4.0.2
attrs                         21.4.0
audioread                     2.1.9
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.6.0
black                         22.1.0
bleach                        4.1.0
branca                        0.4.2
brotlipy                      0.7.0
cachetools                    5.0.0
certifi                       2021.10.8
cffi                          1.15.0
charde

In [28]:
import tensorflow as tf
f = tf.constant([0, 1, 1, 1, 1, 0, 1])
f = tf.squeeze(f)
print(f)

tf.Tensor([0 1 1 1 1 0 1], shape=(7,), dtype=int32)


In [29]:
g = tf.constant([[0], [1], [1], [1], [1], [0], [1]])
g = tf.squeeze(g)
print(g)

tf.Tensor([0 1 1 1 1 0 1], shape=(7,), dtype=int32)
